In [259]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from merge_years.import_data import get_full_data

In [260]:
raw_data = get_full_data('../raw_data')
raw_data

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,yellow_cards,GW,season,position,dreamteam_yearly_average,team_id,team_name,opponent_level,team_level,kickoff_date
0,aaron connolly,0,0,1,0,0.1,534,37,1,0,...,0,4,20,FWD,NaN,4.0,Brighton,1330.0,1010.0,2019-08-31
1,aaron connolly,0,0,1,0,0.3,534,43,1,0,...,0,5,20,FWD,NaN,4.0,Brighton,1180.0,1050.0,2019-09-14
2,aaron connolly,0,0,1,0,4.8,534,58,0,0,...,0,6,20,FWD,NaN,4.0,Brighton,1090.0,1010.0,2019-09-21
3,aaron connolly,0,0,2,0,0.6,534,63,1,0,...,0,7,20,FWD,NaN,4.0,Brighton,1220.0,1010.0,2019-09-28
4,aaron connolly,0,3,53,1,23.8,534,72,0,2,...,0,8,20,FWD,NaN,4.0,Brighton,1240.0,1050.0,2019-10-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114987,ørjan nyland,0,0,0,0,0.0,35,340,0,0,...,0,35,21,GK,0.0,2.0,Aston Villa,1320.0,1100.0,2021-05-09
114988,ørjan nyland,0,0,0,0,0.0,35,180,0,0,...,0,35,21,GK,0.0,2.0,Aston Villa,1130.0,1100.0,2021-05-13
114989,ørjan nyland,0,0,0,0,0.0,35,355,0,0,...,0,36,21,GK,0.0,2.0,Aston Villa,1090.0,1160.0,2021-05-16
114990,ørjan nyland,0,0,0,0,0.0,35,367,0,0,...,0,37,21,GK,0.0,2.0,Aston Villa,1190.0,1160.0,2021-05-19


# Data

## Split dataset

### Train and Test

In [261]:
j=0
for i,row in raw_data.iterrows():
    if row['name']=='caglar söyüncü':
        print(j)
        raw_data['position'].at[i]='DEF'
        j+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [262]:
raw_data['position'].unique()

array(['FWD', 'DEF', 'MID', 'GK'], dtype=object)

In [263]:
ra = 2

test_df = raw_data[(raw_data.season > 20) & (raw_data.GW >= 29-ra)]
train_df = raw_data[~((raw_data.season > 20) & (raw_data.GW >= 29-ra))]

test_df.shape, train_df.shape

((8029, 40), (106963, 40))

### Train and Test Position datasets

In [81]:
test_df_FWD = test_df[(test_df.position == 'FWD')]
test_df_DEF = test_df[(test_df.position == 'DEF')]
test_df_MID = test_df[(test_df.position == 'MID')]
test_df_GK = test_df[(test_df.position == 'GK')]

test_df_FWD.shape, test_df_DEF.shape, test_df_MID.shape, test_df_GK.shape

((1036, 40), (2790, 40), (3275, 40), (928, 40))

In [82]:
train_df_FWD = train_df[(train_df.position == 'FWD')]
train_df_DEF = train_df[(train_df.position == 'DEF')]
train_df_MID = train_df[(train_df.position == 'MID')]
train_df_GK = train_df[(train_df.position == 'GK')]

train_df_FWD.shape, train_df_DEF.shape, train_df_MID.shape, train_df_GK.shape

((15058, 40), (36210, 40), (44058, 40), (11637, 40))

# FWD Preprocessing

## NaN

### FWD Features

In [8]:
train_df_FWD.isnull().sum().sort_values(ascending=False)/len(train_df_FWD) 

dreamteam_yearly_average    0.893741
team_level                  0.665929
opponent_level              0.665929
team_name                   0.665929
team_id                     0.665929
team_a_score                0.000470
team_h_score                0.000470
transfers_out               0.000000
threat                      0.000000
total_points                0.000000
transfers_balance           0.000000
transfers_in                0.000000
name                        0.000000
value                       0.000000
was_home                    0.000000
assists                     0.000000
GW                          0.000000
season                      0.000000
position                    0.000000
yellow_cards                0.000000
selected                    0.000000
saves                       0.000000
goals_scored                0.000000
bonus                       0.000000
bps                         0.000000
clean_sheets                0.000000
creativity                  0.000000
e

In [9]:
# fwd features to be rolled

fwd_features=['name', 'assists', 'bps', 'creativity', 'goals_scored', 
              'influence','minutes','penalties_missed',
              'red_cards','selected','threat','total_points',
              'value','yellow_cards']

# dropped ict index 
# !!!!! To add: kickoff_time, was_home, GW, season, 
# Model TO DO to fill nan: opponent_level, team_level

In [10]:
train_FWD_features = train_df_FWD[fwd_features]
train_FWD_features.shape

test_FWD_features = test_df_FWD[fwd_features]

In [11]:
train_FWD_features.isnull().sum().sort_values(ascending=False)/len(train_FWD_features) 
test_FWD_features.isnull().sum().sort_values(ascending=False)/len(test_FWD_features) 

name                0.0
assists             0.0
bps                 0.0
creativity          0.0
goals_scored        0.0
influence           0.0
minutes             0.0
penalties_missed    0.0
red_cards           0.0
selected            0.0
threat              0.0
total_points        0.0
value               0.0
yellow_cards        0.0
dtype: float64

In [12]:
rolling_train_FWD = train_FWD_features.groupby('name').rolling(ra, closed = 'left').mean()
rolling_test_FWD = test_FWD_features.groupby('name').rolling(ra, closed = 'left').mean()

In [13]:
rolling_train_FWD.rename(columns={'total_points':'rolling_points'},inplace=True)
rolling_test_FWD.rename(columns={'total_points':'rolling_points'},inplace=True)

In [14]:
total_points_train = train_df_FWD['total_points']
total_points_test = test_df_FWD['total_points']

In [15]:
result = map(lambda position:position, total_points_train)
rolling_train_FWD['total_points']=list(result)
result = map(lambda position:position, total_points_test)
rolling_test_FWD['total_points']=list(result)

In [16]:
rolling_train_FWD

assists        bps  creativity  goals_scored  \
name                                                                   
aaron connolly 0            NaN        NaN         NaN           NaN   
               1            NaN        NaN         NaN           NaN   
               2            NaN        NaN         NaN           NaN   
               3       0.000000   1.000000    1.733333      0.000000   
               4       0.000000   1.333333    1.900000      0.000000   
...                         ...        ...         ...           ...   
álvaro negredo 114823  0.000000  13.666667    7.500000      0.333333   
               114824  0.000000  16.333333    7.500000      0.333333   
               114825  0.333333  19.666667   10.166667      0.333333   
               114826  0.333333  18.000000    9.466667      0.333333   
               114827  0.333333  17.666667   20.233333      0.333333   

                       influence    minutes  penalties_missed  red_cards  \
name                                                                       
aaron connolly 0             NaN        NaN               NaN        NaN   
               1             NaN        NaN               NaN        NaN   
               2             NaN        NaN               NaN        NaN   
               3        1.066667  16.666667               0.0        0.0   
               4        1.066667  17.000000               0.0        0.0   
...                          ...        ...               ...        ...   
álvaro negredo 114823  16.600000  67.333333               0.0        0.0   
               114824  17.133333  81.333333               0.0        0.0   
               114825  17.533333  78.333333               0.0        0.0   
               114826  14.666667  79.333333               0.0        0.0   
               114827  17.000000  84.333333               0.0        0.0   

                            selected     threat  rolling_points      value  \
name                                                                         
aaron connolly 0                 NaN        NaN             NaN        NaN   
               1                 NaN        NaN             NaN        NaN   
               2                 NaN        NaN             NaN        NaN   
               3        12277.666667  13.000000        1.000000  45.000000   
               4        23177.333333  13.000000        1.000000  45.000000   
...                              ...        ...             ...        ...   
álvaro negredo 114823  564284.333333  35.000000        3.666667  62.000000   
               114824  624816.333333  30.000000        4.000000  62.666667   
               114825  641492.333333  20.000000        5.000000  62.666667   
               114826  587912.000000  17.333333        5.000000  62.333333   
               114827  526030.000000  28.000000        5.000000  61.666667   

                       yellow_cards  total_points  
name                                               
aaron connolly 0                NaN             1  
               1                NaN             1  
               2                NaN             1  
               3                0.0             1  
               4                0.0            13  
...                             ...           ...  
álvaro negredo 114823           0.0             2  
               114824           0.0            11  
               114825           0.0             2  
               114826           0.0             2  
               114827           0.0             1  

[14907 rows x 14 columns]

In [17]:
# Reset index

train_FWD_data = rolling_train_FWD.reset_index()
test_FWD_data = rolling_test_FWD.reset_index()

In [18]:
# adding train: kickoff_time, was_home, GW, season, 

result = map(lambda gw:gw, train_df_FWD['GW'])
train_FWD_data['GW'] = list(result)

result_1 = map(lambda season:season, train_df_FWD['season'])
train_FWD_data['season'] = list(result_1)

result_2 = map(lambda home:home, train_df_FWD['was_home'])
train_FWD_data['was_home'] = list(result_2)

result_3 = map(lambda kickoff:kickoff, train_df_FWD['kickoff_time'])
train_FWD_data['kickoff_time'] = list(result_3)

# adding test: kickoff_time, was_home, GW, season, 

result = map(lambda gw:gw, test_df_FWD['GW'])
test_FWD_data['GW'] = list(result)

result_1 = map(lambda season:season, test_df_FWD['season'])
test_FWD_data['season'] = list(result_1)

result_2 = map(lambda home:home, test_df_FWD['was_home'])
test_FWD_data['was_home'] = list(result_2)

result_3 = map(lambda kickoff:kickoff, test_df_FWD['kickoff_time'])
test_FWD_data['kickoff_time'] = list(result_3)

In [19]:
# Convert kickoff_time to somthing good

In [20]:
train_FWD_data.drop(columns=['level_1'], inplace=True)
train_fwd_preproc = train_FWD_data.dropna()

test_FWD_data.drop(columns=['level_1'], inplace=True)
test_fwd_preproc = test_FWD_data.dropna()

In [21]:
def get_player_df(df,name):
    return train_fwd_preproc[train_fwd_preproc['name'] == name]

get_player_df(train_fwd_preproc, 'álvaro negredo')

,name,assists,bps,creativity,goals_scored,influence,minutes,penalties_missed,red_cards,selected,threat,rolling_points,value,yellow_cards,total_points,GW,season,was_home,kickoff_time
14872,álvaro negredo,0.666667,20.000000,20.833333,0.333333,25.133333,90.000000,0.0,0.0,6.263427e+05,27.000000,7.000000,66.333333,0.000000,2,4,17,True,14:00:00
14873,álvaro negredo,0.666667,10.333333,19.833333,0.000000,15.666667,90.000000,0.0,0.0,9.309113e+05,23.333333,4.666667,67.666667,0.000000,2,5,17,False,16:30:00
14874,álvaro negredo,0.000000,2.666667,8.133333,0.000000,4.666667,90.000000,0.0,0.0,1.097836e+06,16.333333,2.000000,68.666667,0.000000,1,6,17,True,14:00:00
14875,álvaro negredo,0.000000,1.666667,9.800000,0.000000,4.533333,79.333333,0.0,0.0,1.101161e+06,12.333333,1.666667,68.333333,0.000000,1,7,17,False,14:00:00
14876,álvaro negredo,0.000000,1.666667,3.733333,0.000000,2.000000,57.000000,0.0,0.0,9.649480e+05,4.333333,1.333333,67.333333,0.000000,2,8,17,True,12:30:00
14877,álvaro negredo,0.000000,3.666667,10.700000,0.000000,6.200000,57.000000,0.0,0.0,8.299387e+05,16.000000,1.333333,66.000000,0.000000,2,9,17,False,14:00:00
14878,álvaro negredo,0.000000,2.333333,12.733333,0.000000,8.400000,67.666667,0.0,0.0,7.140777e+05,49.666667,1.666667,65.000000,0.000000,5,10,17,True,14:00:00
14879,álvaro negredo,0.333333,8.333333,21.800000,0.000000,15.800000,88.333333,0.0,0.0,6.327920e+05,57.666667,3.000000,64.333333,0.000000,2,11,17,False,15:00:00
14880,álvaro negredo,0.333333,6.333333,14.733333,0.000000,11.533333,88.333333,0.0,0.0,5.774257e+05,47.333333,3.000000,63.666667,0.000000,2,12,17,True,16:00:00
14881,álvaro negredo,0.333333,8.000000,11.266667,0.000000,8.666667,88.333333,0.0,0.0,5.438527e+05,24.333333,3.000000,63.333333,0.000000,12,13,17,False,15:00:00


In [22]:
test_fwd_preproc.head()

,name,assists,bps,creativity,goals_scored,influence,minutes,penalties_missed,red_cards,selected,threat,rolling_points,value,yellow_cards,total_points,GW,season,was_home,kickoff_time
3,aaron connolly,0.0,1.0,0.133333,0.0,0.333333,20.0,0.0,0.0,23952.333333,13.0,0.666667,52.0,0.0,0,29,21,True,20:00:00
4,aaron connolly,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,23584.333333,0.0,0.000000,52.0,0.0,0,30,21,False,18:30:00
5,aaron connolly,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,23164.666667,0.0,0.000000,52.0,0.0,0,31,21,True,19:15:00
6,aaron connolly,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,22890.333333,0.0,0.000000,52.0,0.0,0,32,21,False,19:15:00
7,aaron connolly,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,22477.333333,0.0,0.000000,52.0,0.0,1,33,21,False,19:00:00


### Encoding

In [23]:
# Encoding was_home
from sklearn.preprocessing import LabelEncoder

# train:
encoder = LabelEncoder()
encoder.fit(train_fwd_preproc[['was_home']])

train_fwd_preproc['was_home'] = encoder.transform(train_fwd_preproc[['was_home']])

# test:
encoder_2 = LabelEncoder()
encoder_2.fit(test_fwd_preproc[['was_home']])

test_fwd_preproc['was_home'] = encoder_2.transform(test_fwd_preproc[['was_home']])

/Users/Stemitsiotis/.pyenv/versions/3.8.6/envs/fflpred/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<ipython-input-23-45467926f5c6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_fwd_preproc['was_home'] = encoder.transform(train_fwd_preproc[['was_home']])
<ipython-input-23-45467926f5c6>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [24]:
train_fwd_preproc

,name,assists,bps,creativity,goals_scored,influence,minutes,penalties_missed,red_cards,selected,threat,rolling_points,value,yellow_cards,total_points,GW,season,was_home,kickoff_time
3,aaron connolly,0.000000,1.000000,1.733333,0.000000,1.066667,16.666667,0.0,0.0,12277.666667,13.000000,1.000000,45.000000,0.0,1,7,20,0,14:00:00
4,aaron connolly,0.000000,1.333333,1.900000,0.000000,1.066667,17.000000,0.0,0.0,23177.333333,13.000000,1.000000,45.000000,0.0,13,8,20,1,11:30:00
5,aaron connolly,0.000000,18.666667,9.733333,0.666667,24.133333,41.333333,0.0,0.0,30176.333333,41.666667,5.000000,45.000000,0.0,1,9,20,0,14:00:00
6,aaron connolly,0.000000,19.333333,11.666667,0.666667,24.933333,49.666667,0.0,0.0,59556.333333,42.666667,5.000000,45.000000,0.0,8,10,20,1,14:00:00
7,aaron connolly,0.666667,26.000000,11.566667,0.666667,24.866667,67.666667,0.0,0.0,97405.666667,43.333333,7.333333,45.333333,0.0,2,11,20,1,15:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14902,álvaro negredo,0.000000,13.666667,7.500000,0.333333,16.600000,67.333333,0.0,0.0,564284.333333,35.000000,3.666667,62.000000,0.0,2,34,17,1,18:45:00
14903,álvaro negredo,0.000000,16.333333,7.500000,0.333333,17.133333,81.333333,0.0,0.0,624816.333333,30.000000,4.000000,62.666667,0.0,11,35,17,1,13:05:00
14904,álvaro negredo,0.333333,19.666667,10.166667,0.333333,17.533333,78.333333,0.0,0.0,641492.333333,20.000000,5.000000,62.666667,0.0,2,36,17,0,19:00:00
14905,álvaro negredo,0.333333,18.000000,9.466667,0.333333,14.666667,79.333333,0.0,0.0,587912.000000,17.333333,5.000000,62.333333,0.0,2,37,17,1,14:00:00


In [25]:
X_train_fwd = train_fwd_preproc.drop(columns=['total_points','kickoff_time', 'name'])
y_train_fwd = train_fwd_preproc['total_points']

X_test_fwd = test_fwd_preproc.drop(columns=['total_points','kickoff_time', 'name'])
y_test_fwd = test_fwd_preproc['total_points']

In [26]:
X_train_fwd

,assists,bps,creativity,goals_scored,influence,minutes,penalties_missed,red_cards,selected,threat,rolling_points,value,yellow_cards,GW,season,was_home
3,0.000000,1.000000,1.733333,0.000000,1.066667,16.666667,0.0,0.0,12277.666667,13.000000,1.000000,45.000000,0.0,7,20,0
4,0.000000,1.333333,1.900000,0.000000,1.066667,17.000000,0.0,0.0,23177.333333,13.000000,1.000000,45.000000,0.0,8,20,1
5,0.000000,18.666667,9.733333,0.666667,24.133333,41.333333,0.0,0.0,30176.333333,41.666667,5.000000,45.000000,0.0,9,20,0
6,0.000000,19.333333,11.666667,0.666667,24.933333,49.666667,0.0,0.0,59556.333333,42.666667,5.000000,45.000000,0.0,10,20,1
7,0.666667,26.000000,11.566667,0.666667,24.866667,67.666667,0.0,0.0,97405.666667,43.333333,7.333333,45.333333,0.0,11,20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14902,0.000000,13.666667,7.500000,0.333333,16.600000,67.333333,0.0,0.0,564284.333333,35.000000,3.666667,62.000000,0.0,34,17,1
14903,0.000000,16.333333,7.500000,0.333333,17.133333,81.333333,0.0,0.0,624816.333333,30.000000,4.000000,62.666667,0.0,35,17,1
14904,0.333333,19.666667,10.166667,0.333333,17.533333,78.333333,0.0,0.0,641492.333333,20.000000,5.000000,62.666667,0.0,36,17,0
14905,0.333333,18.000000,9.466667,0.333333,14.666667,79.333333,0.0,0.0,587912.000000,17.333333,5.000000,62.333333,0.0,37,17,1


In [27]:
X_train_fwd_new = X_train_fwd.drop(columns=['GW', 'season', 'was_home'])
X_test_fwd_new = X_test_fwd.drop(columns=['GW', 'season', 'was_home'])

In [28]:
# Scaling

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train_fwd_new)

MinMaxScaler()

In [29]:
X_train_fwd_new.head()

,assists,bps,creativity,goals_scored,influence,minutes,penalties_missed,red_cards,selected,threat,rolling_points,value,yellow_cards
3,0.000000,1.000000,1.733333,0.000000,1.066667,16.666667,0.0,0.0,12277.666667,13.000000,1.000000,45.000000,0.0
4,0.000000,1.333333,1.900000,0.000000,1.066667,17.000000,0.0,0.0,23177.333333,13.000000,1.000000,45.000000,0.0
5,0.000000,18.666667,9.733333,0.666667,24.133333,41.333333,0.0,0.0,30176.333333,41.666667,5.000000,45.000000,0.0
6,0.000000,19.333333,11.666667,0.666667,24.933333,49.666667,0.0,0.0,59556.333333,42.666667,5.000000,45.000000,0.0
7,0.666667,26.000000,11.566667,0.666667,24.866667,67.666667,0.0,0.0,97405.666667,43.333333,7.333333,45.333333,0.0


In [30]:
X_train_fwd_final = X_train_fwd[['GW', 'season', 'was_home']].reset_index(drop = True)\
        .join(pd.DataFrame(scaler.transform(X_train_fwd_new),
             columns=X_train_fwd_new.columns))

X_test_fwd_final = X_test_fwd[['GW', 'season', 'was_home']].reset_index(drop = True)\
        .join(pd.DataFrame(scaler.transform(X_test_fwd_new),
             columns=X_test_fwd_new.columns))

In [31]:
X_train_fwd_final.head()

,GW,season,was_home,assists,bps,creativity,goals_scored,influence,minutes,penalties_missed,red_cards,selected,threat,rolling_points,value,yellow_cards
0,7,20,0,0.000000,0.090535,0.033965,0.00,0.010811,0.185185,0.0,0.0,0.002635,0.124204,0.10,0.033582,0.0
1,8,20,1,0.000000,0.094650,0.037231,0.00,0.010811,0.188889,0.0,0.0,0.004981,0.124204,0.10,0.033582,0.0
2,9,20,0,0.000000,0.308642,0.190725,0.25,0.244595,0.459259,0.0,0.0,0.006487,0.398089,0.34,0.033582,0.0
3,10,20,1,0.000000,0.316872,0.228609,0.25,0.252703,0.551852,0.0,0.0,0.012809,0.407643,0.34,0.033582,0.0
4,11,20,1,0.333333,0.399177,0.226649,0.25,0.252027,0.751852,0.0,0.0,0.020953,0.414013,0.48,0.037313,0.0


In [32]:
X_test_fwd.head()

,assists,bps,creativity,goals_scored,influence,minutes,penalties_missed,red_cards,selected,threat,rolling_points,value,yellow_cards,GW,season,was_home
3,0.0,1.0,0.133333,0.0,0.333333,20.0,0.0,0.0,23952.333333,13.0,0.666667,52.0,0.0,29,21,1
4,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,23584.333333,0.0,0.000000,52.0,0.0,30,21,0
5,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,23164.666667,0.0,0.000000,52.0,0.0,31,21,1
6,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,22890.333333,0.0,0.000000,52.0,0.0,32,21,0
7,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,22477.333333,0.0,0.000000,52.0,0.0,33,21,0


#### KNN

In [33]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

knn_regressor = KNeighborsRegressor(n_neighbors=7).fit(X_train_fwd_final, y_train_fwd)

base_knn_score = cross_val_score(knn_regressor, X_train_fwd_final, y_train_fwd, cv=10).mean()
base_knn_score

0.15301662402723587

In [34]:
#from sklearn.inspection import permutation_importance
#
#permutation_score = permutation_importance(knn_regressor, X_train_fwd_final, y_train_fwd, n_repeats=10)
#importance_df = pd.DataFrame(np.vstack((X_train_fwd_final.columns,
#                                        permutation_score.importances_mean)).T)
#
#importance_df.columns=['feature','score decrease']
#importance_df.sort_values(by="score decrease", ascending = False)

In [35]:
#k_values=list(range(2, 26))
#score = []
#
#for k in k_values:
#    model = KNeighborsRegressor(n_neighbors=k)
#    s = cross_val_score(model, X_train_fwd_final, y_train_fwd).mean()
#    score.append(s)
#
#plt.plot(k_values, score)

In [36]:
y_pred = knn_regressor.predict(X_test_fwd)
y_pred

array([5.85714286, 3.57142857, 3.57142857, 3.57142857, 3.57142857,
       4.42857143, 4.42857143, 4.42857143, 3.57142857, 3.57142857,
       3.57142857, 3.57142857, 3.57142857, 3.57142857, 3.57142857,
       3.57142857, 3.57142857, 3.57142857, 3.57142857, 3.57142857,
       3.57142857, 3.57142857, 3.57142857, 3.57142857, 3.57142857,
       3.57142857, 3.57142857, 3.57142857, 3.57142857, 3.57142857,
       5.14285714, 5.28571429, 5.28571429, 6.14285714, 6.14285714,
       6.        , 5.85714286, 5.85714286, 5.85714286, 5.85714286,
       6.28571429, 6.28571429, 6.28571429, 7.28571429, 6.28571429,
       6.28571429, 5.28571429, 3.71428571, 4.42857143, 4.42857143,
       5.14285714, 5.85714286, 5.85714286, 3.57142857, 3.57142857,
       3.57142857, 3.57142857, 3.57142857, 3.57142857, 5.85714286,
       5.14285714, 3.57142857, 3.57142857, 3.57142857, 3.57142857,
       3.57142857, 3.57142857, 3.57142857, 3.57142857, 3.57142857,
       5.85714286, 5.28571429, 5.28571429, 3.71428571, 3.71428

In [37]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test_fwd,y_pred)

3.7360350492880614

In [38]:
for i,value in enumerate(y_pred):
    print(y_test_fwd.iloc[i],value)

0 5.857142857142857
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
1 3.5714285714285716
0 4.428571428571429
0 4.428571428571429
0 4.428571428571429
0 3.5714285714285716
1 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
0 3.5714285714285716
1 5.142857142857143
1 5.285714285714286
8 5.285714285714286
2 6.142857142857143
0 6.142857142857143
1 6.0
2 5.857142857142857
0 5.857142857142857
0 5.857142857142857
0 5.857142857142857
11 6.285714285714286
2 6.285714285714286
13 6.285714285714286
2 7.285714285714286
0 6.285714285714286
0 6.285714285714286
1 5.285714285714286
1 3.7142857142857144
0 4.428571428571429
1 4.4

# All positions 

## Preprocessing

In [264]:
train_df_ = train_df.drop(columns=['dreamteam_yearly_average', 'team_level', 'opponent_level', 
                                   'team_name', 'team_id', 'fixture', 'element', 'team_a_score', 'team_h_score', 'transfers_in', 'transfers_out'])
test_df_ = test_df.drop(columns=['dreamteam_yearly_average', 'team_level', 'opponent_level', 
                                 'team_name', 'team_id', 'fixture', 'element', 'team_a_score', 'team_h_score', 'transfers_in', 'transfers_out'])



In [265]:
test_df_

,name,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,...,threat,total_points,transfers_balance,value,was_home,yellow_cards,GW,season,position,kickoff_date
61,aaron connolly,0,0,0,0,0.0,0,0,0.0,0.0,...,0.0,0,-353,52,True,0,27,21,FWD,2021-03-06
62,aaron connolly,0,0,0,0,0.0,0,0,0.0,0.0,...,0.0,0,-858,52,False,0,28,21,FWD,2021-03-14
63,aaron connolly,0,0,0,0,0.0,0,0,0.0,0.0,...,0.0,0,-78,52,True,0,29,21,FWD,2021-03-20
64,aaron connolly,0,0,0,0,0.0,0,0,0.0,0.0,...,0.0,0,-387,52,False,0,30,21,FWD,2021-04-04
65,aaron connolly,0,0,0,0,0.0,0,0,0.0,0.0,...,0.0,0,-423,52,True,0,31,21,FWD,2021-04-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114987,ørjan nyland,0,0,0,0,0.0,0,0,0.0,0.0,...,0.0,0,-2960,40,True,0,35,21,GK,2021-05-09
114988,ørjan nyland,0,0,0,0,0.0,0,0,0.0,0.0,...,0.0,0,-2960,40,True,0,35,21,GK,2021-05-13
114989,ørjan nyland,0,0,0,0,0.0,0,0,0.0,0.0,...,0.0,0,-1358,40,False,0,36,21,GK,2021-05-16
114990,ørjan nyland,0,0,0,0,0.0,0,0,0.0,0.0,...,0.0,0,-1782,40,False,0,37,21,GK,2021-05-19


In [266]:
train_df_rolling = train_df_.drop(columns=['kickoff_time', 'opponent_team',
                                           'was_home', 'GW', 'season', 'position', 
                                           'kickoff_date'])

test_df_rolling = test_df_.drop(columns=['kickoff_time', 'opponent_team',
                                           'was_home', 'GW', 'season', 'position', 
                                           'kickoff_date'])

train_df_rolling = train_df_rolling.groupby('name').rolling(ra, closed = 'left').mean()
test_df_rolling = test_df_rolling.groupby('name').rolling(ra, closed = 'left').mean()

In [267]:
train_df_rolling.rename(columns={'total_points':'rolling_points'},inplace=True)
test_df_rolling.rename(columns={'total_points':'rolling_points'},inplace=True)

total_points_train = train_df_['total_points']
total_points_test = test_df_['total_points']

result = map(lambda position:position, total_points_train)
train_df_rolling['total_points']=list(result)
result = map(lambda position:position, total_points_test)
test_df_rolling['total_points']=list(result)

In [268]:
# Reset index

train_df_rolled = train_df_rolling.reset_index()
test_df_rolled = test_df_rolling.reset_index()

In [ ]:
train_df_rolled

In [269]:
# adding train: kickoff_time, was_home, GW, season, kickoff_date, position, opponent_team

result = map(lambda gw:gw, train_df_['GW'])
train_df_rolled['GW'] = list(result)

result_1 = map(lambda season:season, train_df_['season'])
train_df_rolled['season'] = list(result_1)

result_2 = map(lambda home:home, train_df_['was_home'])
train_df_rolled['was_home'] = list(result_2)

result_3 = map(lambda kickoff:kickoff, train_df_['kickoff_date'])
train_df_rolled['kickoff_date'] = list(result_3)

result_4 = map(lambda kickoff:kickoff, train_df_['kickoff_time'])
train_df_rolled['kickoff_time'] = list(result_4)

result_5 = map(lambda position:position, train_df_['position'])
train_df_rolled['position'] = list(result_5)

#result_6 = map(lambda position:position, train_df_['opponent_team'])
#train_df_rolled['opponent_team'] = list(result_6)

# adding test: kickoff_time, was_home, GW, season, kickoff_date, position, opponent_team 

result = map(lambda gw:gw, test_df_['GW'])
test_df_rolled['GW'] = list(result)

result_1 = map(lambda season:season, test_df_['season'])
test_df_rolled['season'] = list(result_1)

result_2 = map(lambda home:home, test_df_['was_home'])
test_df_rolled['was_home'] = list(result_2)

result_3 = map(lambda kickoff:kickoff, test_df_['kickoff_date'])
test_df_rolled['kickoff_date'] = list(result_3)

result_4 = map(lambda kickoff:kickoff, test_df_['kickoff_time'])
test_df_rolled['kickoff_time'] = list(result_4)

result_5 = map(lambda kickoff:kickoff, test_df_['position'])
test_df_rolled['position'] = list(result_5)

#result_6 = map(lambda kickoff:kickoff, test_df_['opponent_team'])
#test_df_rolled['opponent_team'] = list(result_6)

In [270]:
train_df_rolled.head()

,name,level_1,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,...,transfers_balance,value,yellow_cards,total_points,GW,season,was_home,kickoff_date,kickoff_time,position
0,aaron connolly,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,4,20,False,2019-08-31,14:00:00,FWD
1,aaron connolly,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,5,20,True,2019-09-14,14:00:00,FWD
2,aaron connolly,2,0.0,0.0,1.0,0.0,0.20,1.0,0.0,1.10,...,5294.5,45.0,0.0,1,6,20,False,2019-09-21,16:30:00,FWD
3,aaron connolly,3,0.0,0.0,1.0,0.0,2.55,0.5,0.0,2.35,...,9339.5,45.0,0.0,1,7,20,False,2019-09-28,14:00:00,FWD
4,aaron connolly,4,0.0,0.0,1.5,0.0,2.70,0.5,0.0,1.30,...,8263.5,45.0,0.0,13,8,20,True,2019-10-05,11:30:00,FWD


In [271]:
train_df_rolled.drop(columns=['level_1'], inplace=True)
test_df_rolled.drop(columns=['level_1'], inplace=True)


In [272]:
train_preproc = train_df_rolled.dropna()
test_preproc = test_df_rolled.dropna()

In [273]:
train_preproc.head()

,name,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,...,transfers_balance,value,yellow_cards,total_points,GW,season,was_home,kickoff_date,kickoff_time,position
2,aaron connolly,0.0,0.0,1.0,0.0,0.20,1.0,0.0,1.10,0.6,...,5294.5,45.0,0.0,1,6,20,False,2019-09-21,16:30:00,FWD
3,aaron connolly,0.0,0.0,1.0,0.0,2.55,0.5,0.0,2.35,1.5,...,9339.5,45.0,0.0,1,7,20,False,2019-09-28,14:00:00,FWD
4,aaron connolly,0.0,0.0,1.5,0.0,2.70,0.5,0.0,1.30,1.1,...,8263.5,45.0,0.0,13,8,20,True,2019-10-05,11:30:00,FWD
5,aaron connolly,0.0,1.5,27.5,0.5,12.20,0.5,1.0,10.10,35.2,...,5296.5,45.0,0.0,1,9,20,False,2019-10-19,14:00:00,FWD
6,aaron connolly,0.0,1.5,28.0,0.5,17.20,0.5,1.0,11.85,37.3,...,36479.0,45.0,0.0,8,10,20,True,2019-10-26,14:00:00,FWD


## Encoding

In [274]:
# Encoding was_home
from sklearn.preprocessing import LabelEncoder

# train:
encoder = LabelEncoder()
encoder.fit(train_preproc[['was_home']])

train_preproc['was_home'] = encoder.transform(train_preproc[['was_home']])

# test:
encoder_2 = LabelEncoder()
encoder_2.fit(test_preproc[['was_home']])

test_preproc['was_home'] = encoder_2.transform(test_preproc[['was_home']])

/Users/Stemitsiotis/.pyenv/versions/3.8.6/envs/fflpred/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<ipython-input-274-d97e3b8624fa>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_preproc['was_home'] = encoder.transform(train_preproc[['was_home']])
<ipython-input-274-d97e3b8624fa>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [275]:
# Encoding position

from sklearn.preprocessing import OneHotEncoder

# Train
ohe = OneHotEncoder(sparse=False)
ohe.fit(train_preproc[['position']])

position_encoded = ohe.transform(train_preproc[['position']])

train_preproc["FWD"],train_preproc["DEF"],train_preproc["MID"],train_preproc["GK"]= position_encoded.T

# Test
ohe_2 = OneHotEncoder(sparse=False)
ohe_2.fit(test_preproc[['position']])

position_encoded_2 = ohe_2.transform(test_preproc[['position']])

test_preproc["FWD"],test_preproc["DEF"],test_preproc["MID"],test_preproc["GK"]= position_encoded_2.T

<ipython-input-275-9ca41ca4c396>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_preproc["FWD"],train_preproc["DEF"],train_preproc["MID"],train_preproc["GK"]= position_encoded.T
<ipython-input-275-9ca41ca4c396>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_preproc["FWD"],test_preproc["DEF"],test_preproc["MID"],test_preproc["GK"]= position_encoded_2.T


In [276]:
train_preproc

,name,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,...,GW,season,was_home,kickoff_date,kickoff_time,position,FWD,DEF,MID,GK
2,aaron connolly,0.0,0.0,1.0,0.0,2.000000e-01,1.0,0.0,1.10,6.000000e-01,...,6,20,0,2019-09-21,16:30:00,FWD,0.0,1.0,0.0,0.0
3,aaron connolly,0.0,0.0,1.0,0.0,2.550000e+00,0.5,0.0,2.35,1.500000e+00,...,7,20,0,2019-09-28,14:00:00,FWD,0.0,1.0,0.0,0.0
4,aaron connolly,0.0,0.0,1.5,0.0,2.700000e+00,0.5,0.0,1.30,1.100000e+00,...,8,20,1,2019-10-05,11:30:00,FWD,0.0,1.0,0.0,0.0
5,aaron connolly,0.0,1.5,27.5,0.5,1.220000e+01,0.5,1.0,10.10,3.520000e+01,...,9,20,0,2019-10-19,14:00:00,FWD,0.0,1.0,0.0,0.0
6,aaron connolly,0.0,1.5,28.0,0.5,1.720000e+01,0.5,1.0,11.85,3.730000e+01,...,10,20,1,2019-10-26,14:00:00,FWD,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106958,ørjan nyland,0.0,0.0,0.0,0.0,1.350031e-13,0.0,0.0,0.00,4.263256e-14,...,23,21,1,2021-02-06,12:30:00,GK,0.0,0.0,1.0,0.0
106959,ørjan nyland,0.0,0.0,0.0,0.0,1.350031e-13,0.0,0.0,0.00,4.263256e-14,...,24,21,0,2021-02-13,20:00:00,GK,0.0,0.0,1.0,0.0
106960,ørjan nyland,0.0,0.0,0.0,0.0,1.350031e-13,0.0,0.0,0.00,4.263256e-14,...,25,21,1,2021-02-21,14:05:00,GK,0.0,0.0,1.0,0.0
106961,ørjan nyland,0.0,0.0,0.0,0.0,1.350031e-13,0.0,0.0,0.00,4.263256e-14,...,26,21,0,2021-02-27,17:30:00,GK,0.0,0.0,1.0,0.0


## X and y

In [277]:
X_train = train_preproc.drop(columns=['total_points', 'name', 'position'])
y_train = train_preproc['total_points']

X_test = test_preproc.drop(columns=['total_points', 'name', 'position'])
y_test = test_preproc['total_points']

In [278]:
X_train.head()

,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,minutes,...,yellow_cards,GW,season,was_home,kickoff_date,kickoff_time,FWD,DEF,MID,GK
2,0.0,0.0,1.0,0.0,0.20,1.0,0.0,1.10,0.6,15.0,...,0.0,6,20,0,2019-09-21,16:30:00,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,2.55,0.5,0.0,2.35,1.5,13.0,...,0.0,7,20,0,2019-09-28,14:00:00,0.0,1.0,0.0,0.0
4,0.0,0.0,1.5,0.0,2.70,0.5,0.0,1.30,1.1,22.5,...,0.0,8,20,1,2019-10-05,11:30:00,0.0,1.0,0.0,0.0
5,0.0,1.5,27.5,0.5,12.20,0.5,1.0,10.10,35.2,52.0,...,0.0,9,20,0,2019-10-19,14:00:00,0.0,1.0,0.0,0.0
6,0.0,1.5,28.0,0.5,17.20,0.5,1.0,11.85,37.3,62.0,...,0.0,10,20,1,2019-10-26,14:00:00,0.0,1.0,0.0,0.0


In [279]:
X_train_scale = X_train.drop(columns=['GW', 'season', 'was_home', 'FWD', 'DEF', 'MID', 'GK'])
X_test_scale = X_test.drop(columns=['GW', 'season', 'was_home', 'FWD', 'DEF', 'MID', 'GK'])

# Drop opponent_team???? yes

In [280]:
# Kickoff date 

X_train_scale.drop(columns=['kickoff_time', 'kickoff_date'], inplace=True)
X_test_scale.drop(columns=['kickoff_time', 'kickoff_date'], inplace=True)

In [281]:
# Scaling

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train_scale)

# Transform X_train

X_train_final = X_train[['GW', 'season', 'was_home', 'FWD', 'DEF', 'MID', 'GK']].reset_index(drop = True)\
        .join(pd.DataFrame(scaler.transform(X_train_scale),
             columns=X_train_scale.columns))

In [282]:
# Transform X_test

X_test_final = X_test[['GW', 'season', 'was_home', 'FWD', 'DEF', 'MID', 'GK']].reset_index(drop = True)\
        .join(pd.DataFrame(scaler.transform(X_test_scale),
             columns=X_test_scale.columns))

In [283]:
X_test_final.columns

Index(['GW', 'season', 'was_home', 'FWD', 'DEF', 'MID', 'GK', 'assists',
       'bonus', 'bps', 'clean_sheets', 'creativity', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'minutes', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'saves', 'selected',
       'threat', 'rolling_points', 'transfers_balance', 'value',
       'yellow_cards'],
      dtype='object')

## KNN

In [297]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

knn_regressor_all = KNeighborsRegressor(n_neighbors=23).fit(X_train_final, y_train)

base_knn_score = cross_val_score(knn_regressor_all, X_train_final, y_train).mean()
base_knn_score

0.21860717402578436

In [298]:
from sklearn.metrics import mean_absolute_error

y_pred = knn_regressor_all.predict(X_test_final)

mean_absolute_error(y_test,y_pred)

1.0722993766274758

In [289]:
for i,value in enumerate(y_pred):
    print(y_test.iloc[i],value)

0 0.06666666666666667
0 0.13333333333333333
0 0.3333333333333333
0 0.0
1 0.4
0 0.2
0 0.4666666666666667
0 0.0
0 0.13333333333333333
1 0.2
0 4.8
1 3.066666666666667
0 5.866666666666666
0 1.2
0 1.4666666666666666
2 0.4666666666666667
1 1.9333333333333333
3 1.4666666666666666
5 2.8666666666666667
6 3.1333333333333333
0 0.0
0 0.4
0 0.06666666666666667
0 0.4
0 0.0
0 0.0
0 0.06666666666666667
0 0.13333333333333333
0 0.0
0 0.06666666666666667
4 3.7333333333333334
1 4.266666666666667
3 2.466666666666667
9 4.266666666666667
2 2.933333333333333
3 3.0
10 2.466666666666667
3 3.4
9 2.6666666666666665
2 3.8666666666666667
2 3.8
1 2.1333333333333333
9 0.8666666666666667
5 1.1333333333333333
0 2.7333333333333334
3 1.9333333333333333
2 2.8666666666666667
0 1.5333333333333334
0 0.7333333333333333
0 0.3333333333333333
0 0.2
0 0.06666666666666667
0 0.0
0 0.06666666666666667
3 0.06666666666666667
2 2.466666666666667
3 2.2666666666666666
1 2.6
0 0.2
0 0.26666666666666666
0 0.13333333333333333
0 0.4
0 0.1333

0 0.4666666666666667
0 0.0
0 0.0
0 0.0
0 0.06666666666666667
0 0.4666666666666667
0 0.0
0 0.06666666666666667
0 1.6
0 1.6666666666666667
1 1.2
6 1.2666666666666666
1 1.6666666666666667
8 0.8666666666666667
0 1.8666666666666667
1 2.2
2 1.0666666666666667
8 3.3333333333333335
1 2.3333333333333335
1 1.2666666666666666
2 0.4
2 1.8666666666666667
2 2.2
11 4.866666666666666
9 3.2
1 3.466666666666667
0 3.933333333333333
0 0.13333333333333333
0 0.3333333333333333
0 0.3333333333333333
0 0.0
0 0.06666666666666667
0 0.3333333333333333
0 0.0
0 0.13333333333333333
0 0.7333333333333333
2 1.8
1 2.6666666666666665
2 1.8
1 2.4
3 1.7333333333333334
1 2.3333333333333335
1 2.8666666666666667
2 1.4
2 2.066666666666667
0 2.066666666666667
0 2.7333333333333334
0 0.4666666666666667
0 0.5333333333333333
0 0.0
1 2.4
5 1.6666666666666667
2 3.066666666666667
5 1.4
0 1.8666666666666667
0 0.4666666666666667
0 0.9333333333333333
0 0.13333333333333333
0 0.06666666666666667
0 0.2
0 0.2
0 0.06666666666666667
0 0.0
10 2

3 1.9333333333333333
2 1.8
0 2.6
2 1.7333333333333334
2 1.3333333333333333
0 2.2
0 0.0
0 0.5333333333333333
0 0.06666666666666667
0 0.0
0 0.0
0 0.0
0 0.06666666666666667
0 0.4666666666666667
0 0.0
0 0.0
0 0.0
0 0.13333333333333333
0 0.13333333333333333
0 0.6666666666666666
0 0.06666666666666667
0 0.0
0 -0.06666666666666667
0 -0.06666666666666667
0 0.06666666666666667
0 0.0
0 0.06666666666666667
7 0.4666666666666667
8 2.4
2 2.3333333333333335
1 3.2
2 2.466666666666667
1 4.133333333333334
5 4.866666666666666
1 3.066666666666667
6 4.133333333333334
0 0.3333333333333333
0 1.2
0 1.6666666666666667
2 1.8
0 2.2666666666666666
6 1.2666666666666666
2 1.8
7 0.4666666666666667
2 2.8
6 3.0
1 0.0
1 0.4
8 1.0
2 2.8
1 3.8666666666666667
6 1.8666666666666667
1 1.6
0 2.2
6 1.0
1 1.0666666666666667
0 0.0
0 0.06666666666666667
0 0.4666666666666667
0 0.0
0 0.0
0 0.0
0 0.06666666666666667
0 0.4666666666666667
1 0.0
7 0.06666666666666667
0 2.6
1 2.1333333333333333
1 0.5333333333333333
0 0.26666666666666666


0 0.0
0 0.06666666666666667
0 0.06666666666666667
0 0.06666666666666667
0 0.6
0 0.06666666666666667
0 0.0
0 2.6
0 0.7333333333333333
1 0.0
9 0.3333333333333333
2 3.7333333333333334
6 2.7333333333333334
6 3.0
2 3.533333333333333
7 4.866666666666666
2 2.4
0 0.13333333333333333
0 0.13333333333333333
0 0.0
0 0.0
0 0.06666666666666667
0 0.06666666666666667
0 0.13333333333333333
0 0.0
0 0.06666666666666667
0 0.0
2 2.8
3 1.4666666666666666
2 1.8666666666666667
3 2.1333333333333333
0 2.6666666666666665
1 1.2666666666666666
2 0.5333333333333333
10 1.6666666666666667
0 2.8
0 0.9333333333333333
0 0.06666666666666667
0 0.06666666666666667
0 0.9333333333333333
0 0.2
0 0.4666666666666667
0 0.4
0 0.9333333333333333
0 0.06666666666666667
2 1.2666666666666666
6 1.6666666666666667
0 1.2666666666666666
6 0.8666666666666667
6 0.2
0 0.6
8 1.4
5 2.6
11 3.2
8 3.4
0 0.0
0 0.2
0 0.0
0 0.13333333333333333
0 0.0
0 0.0
0 0.0
0 0.06666666666666667
0 0.06666666666666667
2 2.8666666666666667
1 3.0
2 1.73333333333333

1 0.7333333333333333
0 0.06666666666666667
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
7 2.7333333333333334
1 3.066666666666667
0 2.933333333333333
0 0.5333333333333333
0 0.13333333333333333
13 0.2
2 4.8
2 4.666666666666667
9 2.933333333333333
2 4.666666666666667
0 0.8666666666666667
0 0.13333333333333333
0 0.13333333333333333
0 0.6666666666666666
0 0.0
0 0.0
0 0.06666666666666667
0 0.26666666666666666
0 0.4666666666666667
0 0.26666666666666666
6 0.2
0 0.4
0 0.4
0 0.2
0 0.3333333333333333
1 0.3333333333333333
0 0.06666666666666667
0 0.26666666666666666
0 0.7333333333333333
2 0.13333333333333333
4 1.6
2 1.8
3 2.6
2 1.7333333333333334
1 2.3333333333333335
2 1.5333333333333334
2 1.7333333333333334
3 2.1333333333333333
3 2.533333333333333
0 0.4
0 0.06666666666666667
0 0.0
0 0.0
0 0.0
0 0.0
0 0.2
1 0.26666666666666666
1 1.4666666666666666
2 1.4666666666666666
0 1.4
11 0.8666666666666667
2 2.1333333333333333
3 2.466666666666667
1 2.4
0 2.0
8 2.4
7 4.133333333333334
1 6.066666666666666
9 

1 2.3333333333333335
1 2.8666666666666667
1 2.533333333333333
5 1.8
6 2.8666666666666667
0 0.5333333333333333
6 1.6666666666666667
2 2.933333333333333
1 1.8666666666666667
0 0.4
0 0.13333333333333333
0 0.0
0 0.06666666666666667
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.06666666666666667
0 0.0
0 0.6
0 0.0
0 0.0
0 0.06666666666666667
0 0.13333333333333333
0 0.4666666666666667
0 0.26666666666666666
0 0.0
0 0.06666666666666667
0 0.0
0 0.4
0 0.06666666666666667
0 0.0
0 0.0
0 0.0
0 0.0
0 0.06666666666666667
0 0.06666666666666667
0 0.13333333333333333
0 0.0
0 0.13333333333333333
0 0.3333333333333333
0 0.06666666666666667
0 0.0
0 0.13333333333333333
0 0.2
0 0.13333333333333333
0 1.8666666666666667
0 0.4
0 0.9333333333333333
0 0.9333333333333333
0 0.0
0 0.2
0 0.4666666666666667
0 0.9333333333333333
0 0.06666666666666667
0 0.0
0 0.06666666666666667
0 0.0
0 0.13333333333333333
0 0.0
0 0.06666666666666667
0 0.06666666666666667
0 0.06666666666666667
1 0.0
0 1.7333333333333334
0 0.06666666666666667
0 

In [301]:
#k_values=list(range(10, 20))
#score = []
#
#for k in k_values:
#    model = KNeighborsRegressor(n_neighbors=k)
#    s = cross_val_score(model, X_train_final, y_train).mean()
#    score.append(s)
#
#plt.plot(k_values, score)

In [302]:
#from sklearn.inspection import permutation_importance
#
#permutation_score = permutation_importance(knn_regressor_all, X_train_final, y_train)
#importance_df = pd.DataFrame(np.vstack((X_train_final.columns,
#                                        permutation_score.importances_mean)).T)
#
#importance_df.columns=['feature','score decrease']
#importance_df.sort_values(by="score decrease", ascending = False)